<a href="https://colab.research.google.com/github/ppg18887/Demo1/blob/master/Keras_AutoML_data_for_Image_and_Structured_Input.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install autokeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 162 kB 5.3 MB/s 
     |████████████████████████████████| 135 kB 35.9 MB/s 
     |████████████████████████████████| 1.6 MB 45.8 MB/s 


In [2]:
import numpy as np

import autokeras as ak

In [8]:
num_instances = 100
# Generate image data.
image_data = np.random.rand(num_instances, 32, 32, 3).astype(np.float32)
# Generate structured data.
structured_data = np.random.rand(num_instances, 20).astype(np.float32)
print(image_data)


[[[[0.09714546 0.9063582  0.84231615]
   [0.48706585 0.257094   0.13009845]
   [0.47093487 0.30598986 0.03063761]
   ...
   [0.3703056  0.1363163  0.37962088]
   [0.217356   0.06297014 0.6253212 ]
   [0.6879561  0.93754655 0.62652206]]

  [[0.02122825 0.04137213 0.3776427 ]
   [0.7092177  0.11330537 0.19529769]
   [0.4753545  0.272236   0.6325024 ]
   ...
   [0.71396255 0.7796073  0.5035612 ]
   [0.769995   0.62050664 0.9077472 ]
   [0.12736031 0.8340955  0.7136245 ]]

  [[0.1725233  0.08606758 0.65359604]
   [0.6723727  0.6790918  0.25973964]
   [0.408459   0.4628479  0.91194254]
   ...
   [0.9071972  0.636017   0.29624382]
   [0.9337911  0.2455005  0.18717256]
   [0.53864056 0.7629807  0.9733936 ]]

  ...

  [[0.7200565  0.26248047 0.39571467]
   [0.5745716  0.5465776  0.26329333]
   [0.1323758  0.95843714 0.56279063]
   ...
   [0.26332653 0.84748816 0.16709843]
   [0.6197694  0.7236778  0.37706134]
   [0.9321676  0.58824193 0.33374536]]

  [[0.4707145  0.30490986 0.36971438]
   [0.4

In [9]:
# Generate regression targets.
regression_target = np.random.rand(num_instances, 1).astype(np.float32)
# Generate classification labels of five classes.
classification_target = np.random.randint(5, size=num_instances)
print(classification_target)

[1 1 3 0 1 2 2 4 1 3 4 2 3 0 4 0 2 2 1 3 3 4 1 4 0 2 3 1 0 0 0 4 2 0 3 3 1
 3 2 4 0 1 3 2 4 4 2 4 0 0 3 1 1 1 2 1 0 3 4 3 2 3 1 3 1 2 2 1 4 4 4 4 3 4
 0 1 3 4 4 4 3 2 0 4 1 2 1 0 4 3 1 1 2 4 0 1 0 2 0 0]


In [5]:
# Initialize the multi with multiple inputs and outputs.
model = ak.AutoModel(
    inputs=[ak.ImageInput(), ak.StructuredDataInput()],
    outputs=[
        ak.RegressionHead(metrics=["mae"]),
        ak.ClassificationHead(loss="categorical_crossentropy", metrics=["accuracy"]),
    ],
    overwrite=True,
    max_trials=2,
)
# Fit the model with prepared data.
model.fit(
    [image_data, structured_data],
    [regression_target, classification_target],
    epochs=3,
)

Trial 2 Complete [00h 00m 40s]
val_loss: 2.075404644012451

Best val_loss So Far: 1.9000447988510132
Total elapsed time: 00h 01m 20s
Epoch 1/3
4/4 [==============================] - 16s 2s/step - loss: 10.5398 - regression_head_1_loss: 7.8565 - classification_head_1_loss: 2.6833 - regression_head_1_mae: 2.1507 - classification_head_1_accuracy: 0.1800
Epoch 2/3
4/4 [==============================] - 8s 2s/step - loss: 19.2182 - regression_head_1_loss: 16.9509 - classification_head_1_loss: 2.2673 - regression_head_1_mae: 3.2134 - classification_head_1_accuracy: 0.3900
Epoch 3/3
4/4 [==============================] - 8s 2s/step - loss: 19.2954 - regression_head_1_loss: 17.0649 - classification_head_1_loss: 2.2306 - regression_head_1_mae: 3.4189 - classification_head_1_accuracy: 0.3800


In [6]:

#Validation data with last 15%
model.fit(
    [image_data, structured_data],
    [regression_target, classification_target],
    # Split the training data and use the last 15% as validation data.
    validation_split=0.15,
    epochs=2,
)


In [7]:
#Customized search space 
input_node1 = ak.ImageInput()
output_node = ak.Normalization()(input_node1)
output_node = ak.ImageAugmentation()(output_node)
output_node1 = ak.ConvBlock()(output_node)
output_node2 = ak.ResNetBlock(version="v2")(output_node)
output_node1 = ak.Merge()([output_node1, output_node2])

input_node2 = ak.StructuredDataInput()
output_node = ak.CategoricalToNumerical()(input_node2)
output_node2 = ak.DenseBlock()(output_node)

output_node = ak.Merge()([output_node1, output_node2])
output_node1 = ak.ClassificationHead()(output_node)
output_node2 = ak.RegressionHead()(output_node)

auto_model = ak.AutoModel(
    inputs=[input_node1, input_node2],
    outputs=[output_node1, output_node2],
    overwrite=True,
    max_trials=2,
)

image_data = np.random.rand(num_instances, 32, 32, 3).astype(np.float32)
structured_data = np.random.rand(num_instances, 20).astype(np.float32)
regression_target = np.random.rand(num_instances, 1).astype(np.float32)
classification_target = np.random.randint(5, size=num_instances)

auto_model.fit(
    [image_data, structured_data],
    [classification_target, regression_target],
    batch_size=32,
    epochs=3,
)


Trial 2 Complete [00h 03m 45s]
val_loss: 1.7502562999725342

Best val_loss So Far: 1.682603120803833
Total elapsed time: 00h 04m 33s
Epoch 1/3
4/4 [==============================] - 16s 2s/step - loss: 3.3753 - classification_head_1_loss: 1.9173 - regression_head_1_loss: 1.4581 - classification_head_1_accuracy: 0.2200 - regression_head_1_mean_squared_error: 1.4581
Epoch 2/3
4/4 [==============================] - 8s 2s/step - loss: 4.1646 - classification_head_1_loss: 1.8538 - regression_head_1_loss: 2.3107 - classification_head_1_accuracy: 0.2600 - regression_head_1_mean_squared_error: 2.3107
Epoch 3/3
4/4 [==============================] - 8s 2s/step - loss: 5.0858 - classification_head_1_loss: 1.8238 - regression_head_1_loss: 3.2619 - classification_head_1_accuracy: 0.2100 - regression_head_1_mean_squared_error: 3.2619
